In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Feb  5 20:38:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import torch
torch.__version__

'1.4.0'

In [4]:
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()  # disable dropout (or leave in train mode to finetune)

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/fairseq
creating build/temp.linux-x86_64-3.6/fairseq/clib
creating build/temp.linux-x86_64-3.6/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.6/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=libbleu -D_GLIBCXX_USE_CXX11_ABI=0
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python

100%|██████████| 3699866548/3699866548 [01:57<00:00, 31475533.18B/s]
1042301B [00:00, 2338298.47B/s]
456318B [00:00, 13160171.71B/s]


BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [0]:
tokens = bart.encode('Hello world!')
print(tokens.tolist())
bart.decode(tokens)  # 'Hello world!'

[0, 31414, 232, 328, 2]


'Hello world!'

In [0]:
from fairseq.data.data_utils import collate_tokens

batch_of_sentences = [
    'BART is a seq2seq model.',
    'BART is denoising autoencoder.'
]

batch = collate_tokens(
    [bart.encode(s) for s in batch_of_sentences], pad_idx=1
)

last_layer_features = bart.extract_features(batch)
# assert last_layer_features.size() == torch.Size([1, 5, 1024])

In [0]:
batch

tensor([[    0,   387, 11328,    16,    10, 48652,   176, 47762,  1421,     4,
             2,     1,     1],
        [    0,   387, 11328,    16,  3069,   139,  3009,  7241, 18057,   438,
         15362,     4,     2]])

In [0]:
batch.shape

torch.Size([2, 13])

In [0]:
last_layer_features.shape

torch.Size([2, 13, 1024])

## Convert whole dataset to bart embeddings

In [0]:
import pandas as pd

In [0]:
PATH = '/content/drive/My Drive/Kaggle/Google QUEST Q&A Labeling/data/base/'
OUTPUT_PATH = '/content/drive/My Drive/Kaggle/Google QUEST Q&A Labeling/output/2/'

In [7]:
df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [0]:
df_train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [0]:
BART_BATCH_SIZE = 16
MAX_TITLE_LENGTH = 64
MAX_QUESTION_LENGTH = 512
MAX_ANSWER_LENGTH = 512
MAX_TITLE_QUESTION_LENGTH = MAX_TITLE_LENGTH + MAX_QUESTION_LENGTH

In [0]:
from fairseq.data.data_utils import collate_tokens

def extract_features_for_batch(batch_snt, max_length):
  batch_encoded = []
  trimmed_count = 0
  for s in batch_snt:
    encoded = bart.encode(s)
    if len(encoded) <= max_length:
      encoded = torch.cat((encoded, torch.tensor([1]).repeat(max_length - len(encoded))), 0)
    else:
      encoded = encoded[:max_length]
      trimmed_count += 1
    batch_encoded.append(encoded)
  batch = collate_tokens(batch_encoded, pad_idx=1)
  # print('trimmed_count:', trimmed_count)
  return batch

In [43]:
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

titles = []
questions = []
answers = []
data_q = torch.zeros([0, BART_BATCH_SIZE, MAX_TITLE_QUESTION_LENGTH], dtype=torch.int64)
data_a = torch.zeros([0, BART_BATCH_SIZE, MAX_ANSWER_LENGTH], dtype=torch.int64)
for index, row in tqdm(df_train.iterrows()):
  titles.append(row['question_title'])
  questions.append(row['question_body'])
  answers.append(row['answer'])
  if len(titles) < BART_BATCH_SIZE:
    continue
  
  titles_f = extract_features_for_batch(titles, MAX_TITLE_LENGTH)
  questions_f = extract_features_for_batch(questions, MAX_QUESTION_LENGTH)
  answers_f = extract_features_for_batch(answers, MAX_ANSWER_LENGTH)
  feat_q = torch.cat((titles_f, questions_f), 1).unsqueeze(0)
  feat_a = answers_f.unsqueeze(0)

  data_q = torch.cat((data_q, feat_q), 0)
  data_a = torch.cat((data_a, feat_a), 0)

  titles = []
  questions = []
  answers = []
# TODO: extract the rest?

data_q = data_q.contiguous().to(device)
data_a = data_a.contiguous().to(device)
print(data_q.shape)
print(data_a.shape)

6079it [00:39, 152.58it/s]

torch.Size([379, 16, 576])
torch.Size([379, 16, 512])


## Build pytorch model

In [0]:
import math
import torch
import torch.nn.functional as F
import torch.nn as nn

class BartMultiLabelModel(nn.Module):

    def __init__(self, bart, input_q_size, input_a_size, labels_count):
        super(BartMultiLabelModel, self).__init__()
        
        self.bart = bart
        self.labels_count = labels_count
        self.b2l = nn.Linear(input_q_size + input_a_size, labels_count)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.b2l.weight.data.uniform_(-initrange, initrange)

    def forward(self, src_q, src_a):
        features_q = self.bart.extract_features(src_q)
        features_a = self.bart.extract_features(src_a)
        features = torch.cat((features_q, features_a), 1)
        print('feat.shape:', features.shape)
        labels_logits = self.b2l(features)
        labels = torch.sigmoid(labels_logits)
        return labels

## Initiate an instance

In [0]:
dropout = 0.2 # the dropout value
model = BartMultiLabelModel(bart, emsize, nhead, nhid, nlayers, dropout).to(device)

## Run the model

In [0]:
criterion = nn.BCELoss()
lr = 2e-5 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

